# EDA on the 13 Node
Notebook designed to explore the methods and attributes of DSS

Author: Javier SaO

Date: 2022-06-08

In [1]:
import dss as dss_interface # Because is a little faster than opendssdirect
import os
import pathlib # Paths become platform agnostic
import re
from dss_controller import Controller
import numpy as np
import pandas as pd

np.set_printoptions(precision=3, suppress=True)

dss = dss_interface.DSS
dss.Start(0)

# "Robust" way of handling paths
path_proj = os.path.abspath('')
file_Circuit = pathlib.Path(path_proj).joinpath("13Bus", "IEEE13Nodeckt.dss")
file_Coords = pathlib.Path(path_proj).joinpath("13Bus", "IEEE13Node_BusXY.csv")

#file_13Bus = '13Bus/IEEE13Nodeckt.dss'

dss_text = dss.Text
dss_circuit = dss.ActiveCircuit

dss_text.Command = "clear"
dss_text.Command = f"compile {file_Circuit}"
dss_text.Command = "New EnergyMeter.Feeder Transformer.sub 1" # Used to calculate distance of busses
dss_circuit.Solution.Solve()

In [2]:
dss_controller = Controller(dss_circuit)
dss_controller.print_busNames()

['sourcebus', '650', 'rg60', '633', '634', '671', '645', '646', '692', '675', '611', '652', '670', '632', '680', '684']


In [3]:
# Show Cicuit Elements
print("\nAll Element Names")
print("num--\tphases\tenabled\tname--\t\tbusNames--")
n = dss_circuit.NumCktElements
for i in range(n):
    elem = dss_circuit.CktElements(i)
    print(f"{i+1}/{n}\t{elem.NumPhases}\t{elem.Enabled}\t{elem.Name}\t{elem.BusNames}")
    
# Element types
ckt_types = []
i = 0
print("\nAll Types and Property Names")
for j in range(n):
    elem = dss_circuit.CktElements(j)
    elem_type = re.sub(r"\..*", "", elem.Name)
    if elem_type not in ckt_types:
        ckt_types.append(elem_type)
        print(f"{i}: {elem_type}: {sorted(elem.AllPropertyNames)}")
        i+=1
    
print("\nNothing in Variable Names and Variable Values")


All Element Names
num--	phases	enabled	name--		busNames--
1/39	3	True	Vsource.source	['sourcebus', 'sourcebus.0.0.0']
2/39	3	True	Transformer.sub	['sourcebus', '650']
3/39	1	True	Transformer.reg1	['650.1', 'rg60.1']
4/39	1	True	RegControl.reg1	['rg60.1']
5/39	1	True	Transformer.reg2	['650.2', 'rg60.2']
6/39	1	True	RegControl.reg2	['rg60.2']
7/39	1	True	Transformer.reg3	['650.3', 'rg60.3']
8/39	1	True	RegControl.reg3	['rg60.3']
9/39	3	True	Transformer.xfm1	['633', '634']
10/39	3	True	Load.671	['671.1.2.3']
11/39	1	True	Load.634a	['634.1']
12/39	1	True	Load.634b	['634.2']
13/39	1	True	Load.634c	['634.3']
14/39	1	True	Load.645	['645.2']
15/39	1	True	Load.646	['646.2.3']
16/39	1	True	Load.692	['692.3.1']
17/39	1	True	Load.675a	['675.1']
18/39	1	True	Load.675b	['675.2']
19/39	1	True	Load.675c	['675.3']
20/39	1	True	Load.611	['611.3']
21/39	1	True	Load.652	['652.1']
22/39	1	True	Load.670a	['670.1']
23/39	1	True	Load.670b	['670.2']
24/39	1	True	Load.670c	['670.3']
25/39	3	True	Capacitor.cap1

In [101]:
# Buses

In [4]:
# Show Cicuit Elements
print("\nAllBusNames")
busNames  = dss_circuit.AllBusNames
busVolts  = np.around(dss_circuit.AllBusVolts , 3)  # Returns all Voltages in a single array, does not differentaite between buses. I would not use this
print(busNames )
print(busVolts )

print("\nAll buses by index")
n = len(busNames)
for i in range(n):
    ubus = dss_circuit.Buses(i)
    ubusVoltages = np.around(ubus.Voltages, 3)
    ubusVmag = np.around(ubus.VMagAngle, 3)
    ubusPuVmag = np.around(ubus.puVmagAngle, 3)
    ubuskVBase = np.around(ubus.kVBase, 3)
    
    print(f"{i+1}/{n}: {ubus.Name}")
    print(f"\tVolts    {ubusVoltages}")
    print(f"\tVmag     {ubusVmag}")
    print(f"\tpuVmag   {ubusPuVmag}")
    print(f"\tkVBase   {ubuskVBase}")
    
    if i > 5:
        print("... I've seen enough. I'm satisfied")
        break
# Note: Vmag = kVBase * puVMag


AllBusNames
['sourcebus', '650', 'rg60', '633', '634', '671', '645', '646', '692', '675', '611', '652', '670', '632', '680', '684']
[ 57502.687  33189.476    -10.988 -66394.869 -57491.698  33205.393
   2401.563     -0.467  -1201.238  -2079.718  -1200.312   2080.142
   2536.356     -0.579  -1246.26   -2157.488  -1267.588   2196.936
   2426.426   -109.959  -1300.021  -2096.277  -1120.437   2128.615
    273.121    -15.652   -149.221   -236.287   -124.741    242.009
   2350.079   -221.069  -1338.409  -2109.789  -1015.427   2083.132
  -1295.688  -2078.359  -1122.401   2129.571  -1296.245  -2073.152
  -1121.802   2124.363  -1015.427   2083.132   2350.079   -221.069
  -1338.409  -2109.789   2333.501   -229.744  -1347.987  -2110.401
  -1013.988   2078.666  -1002.147   2078.77    2332.428   -217.299
   2407.055   -145.364  -1312.305  -2102.363  -1083.015   2116.12
   2433.85    -107.516  -1300.765  -2101.262  -1123.577   2134.148
   2350.079   -221.069  -1338.409  -2109.789  -1015.427   2083.1

In [5]:
# Iterating over loads

# Method inspired by Moosa
# Active Ckt Element index starts at 1, and becomes 0 when there are none left

# Example definition in *.dss file
# New Load.671 Bus1=671.1.2.3  Phases=3 Conn=Delta Model=1 kV=4.16   kW=1155 kvar=660 
def read_propsLoad(dss_circuit, do_print = False):
    loadIdx = dss_circuit.Loads.First
    loadCtr = 0
    dictLoads = {"idx": [],
                 "name": [],
                 "phases": [],
                 "kv": [],
                 "kw": [],
                 "kvar": [],
                 "volts": [],
                 "current": [],
                 "power": [],
                 "voltsabs": [],
                 "currentabs": [],
                 "powerabs": [],
                }

    while loadIdx != 0:
        lName = dss_circuit.ActiveCktElement.Name
        lPhases = dss_circuit.ActiveCktElement.NumPhases
        lBus = dss_circuit.ActiveCktElement.BusNames

        # print(dss_circuit.ActiveCktElement.AllPropertyNames)
        lkw = dss_circuit.Loads.kW
        lkvar = dss_circuit.Loads.kvar
        lkv = dss_circuit.Loads.kV


        lVoltsRaw = dss_circuit.ActiveCktElement.Voltages
        lCurrentRaw = dss_circuit.ActiveCktElement.Currents
        lPowerRaw = dss_circuit.ActiveCktElement.Powers

        #lVolts = np.array([complex(real, im) for real, im in zip(lVoltsRaw[:2*lPhases:2], lVoltsRaw[1:2*lPhases:2])])
        lVolts = np.array([complex(real, im) for real, im in zip(lVoltsRaw[::2], lVoltsRaw[1::2])])
        lCurrent = np.array([complex(real, im) for real, im in zip(lCurrentRaw[:2*lPhases:2], lCurrentRaw[1:2*lPhases:2])])
        lPower = np.array([complex(real, im) for real, im in zip(lPowerRaw[:2*lPhases:2], lPowerRaw[1:2*lPhases:2])])

        lVoltsAbs   = abs(lVolts)
        lCurrentAbs = abs(lCurrent)
        lPowerAbs   = abs(lPower)
        
        if do_print:
            print(f"{loadIdx} {lName}; Phases: {lPhases}")
            print(f"\tProps \tkV {lkv}\tkW {lkw}\tkvar {lkvar}")
            print(f"\tVolts \t{np.around(lVolts,2)}\t{np.around(lVoltsAbs, 2)}")
            print(f"\tPower \t{np.around(lPower,2)}\t{np.around(lPowerAbs, 2)}")
            print(f"\tBuses \t{lBus}")
            #print(f"\tCurrent\t{np.around(lCurrent,2)}\t{np.around(lCurrentAbs, 2)}")
            #print(f"\tPower \t{np.around(lPower,2)}\t{np.around(lPowerAbs, 2)}")
            #print()
        
        dictLoads["idx"].append(loadIdx)
        dictLoads["name"].append(lName)
        dictLoads["phases"].append(lPhases)

        dictLoads["kv"].append(lkv)
        dictLoads["kw"].append(lkw)
        dictLoads["kvar"].append(lkvar)

        dictLoads["volts"].append(lVolts)
        dictLoads["current"].append(lCurrent)
        dictLoads["power"].append(lPower)

        dictLoads["voltsabs"].append(lVoltsAbs)
        dictLoads["currentabs"].append(lCurrentAbs)
        dictLoads["powerabs"].append(lPowerAbs)

        loadIdx = dss_circuit.Loads.Next

    df_Loads = pd.DataFrame.from_dict(dictLoads)
    
    if do_print:
        display(df_Loads.head())
    return df_Loads


In [6]:
#df_Loads1 = read_propsLoad(dss_circuit)
#df_Loads1[["name", "kv", "kw", "kvar"]].head()
#df_Loads1.head()

df_Loads1 = dss_controller.read_propsLoad()
#dss_controller.df_Loads[["name", "kv", "kw", "kvar"]].head()
df_Loads1[["name", "kv", "kw", "kvar"]].head()

,name,kv,kw,kvar
0,Load.671,4.160,1155.0,660.0
1,Load.634a,0.277,160.0,110.0
2,Load.634b,0.277,120.0,90.0
3,Load.634c,0.277,120.0,90.0
4,Load.645,2.400,170.0,125.0


In [83]:
#loadchangecommand = 'Edit Load.' + str(nodeNumber) + ' kW= ' + str(p[loadIdx]) + ' kvar= ' + str(q[loadIdx])
#dssText.Command = loadchangecommand
#dssText.Command = 'Set mode=snapshot'
#dssText.Command = 'solve'

loadchangecommand = "Edit Load.634a kW = 165 kvar=115"
dss_text.Command = loadchangecommand
dss_text.Command = 'set mode=snapshot'
dss_text.Command = 'solve'

In [84]:
df_Loads2 = dss_controller.read_propsLoad()
df_Loads2[["name", "kv", "kw", "kvar"]].head()
#dss_controller.df_Loads[["name", "kv", "kw", "kvar"]].head()

,name,kv,kw,kvar
0,Load.671,4.160,1155.0,660.0
1,Load.634a,0.277,165.0,115.0
2,Load.634b,0.277,120.0,90.0
3,Load.634c,0.277,120.0,90.0
4,Load.645,2.400,170.0,125.0


In [85]:
#df_Loads2[["name", "kv", "kw", "kvar"]].head()

In [99]:
#dss_text.Command = 'set mode=snapshot'
dss_text.Command = 'solve'
df_Loads2 = read_propsLoad(dss_circuit)
df_Loads2[["name", "kv", "kw", "kvar", "volts"]].head()

,name,kv,kw,kvar,volts
0,Load.671,4.160,1155.0,660.0,"[(2348.7554923753455-221.55162653989336j), (-1..."
1,Load.634a,0.277,165.0,115.0,"[(272.6509142076713-15.777882415047555j), 0j]"
2,Load.634b,0.277,120.0,90.0,"[(-149.30328413854764-236.32417611033782j), 0j]"
3,Load.634c,0.277,120.0,90.0,"[(-124.81670464785688+241.98376513948344j), 0j]"
4,Load.645,2.400,170.0,125.0,"[(-1296.2809044643025-2078.621494857307j), 0j]"


In [13]:
df_Loads1[["name", "kv", "kw", "kvar", "volts"]].head()

,name,kv,kw,kvar,volts
0,Load.671,4.160,1155.0,660.0,"[(2350.0786122786512-221.06904907887406j), (-1..."
1,Load.634a,0.277,160.0,110.0,"[(273.121141080331-15.652357543022138j), 0j]"
2,Load.634b,0.277,120.0,90.0,"[(-149.22124909743232-236.28695774995862j), 0j]"
3,Load.634c,0.277,120.0,90.0,"[(-124.74067151469185+242.00907089788836j), 0j]"
4,Load.645,2.400,170.0,125.0,"[(-1295.6881026261979-2078.3589232339277j), 0j]"


## Experiments

In [28]:
dict1 = {"name": ["load1", "load2"],
         "type": ["load", "load"],
         "volts": [10, 12],
         "current": [2, 1.8]
        }
dict2 = {"name": ["bus1", "bus2", "bus3"],
         "type": ["bus", "bus", "bus"],
         "volts": [10.1, 12.1, 0],
         "x": [1, 2, 3],
         "y": [4, 5, 6],
        }

df_1 = pd.DataFrame.from_dict(dict1)
df_2 = pd.DataFrame.from_dict(dict2)

display(df_1)
display(df_2)

,name,type,volts,current
0,load1,load,10,2.0
1,load2,load,12,1.8


,name,type,volts,x,y
0,bus1,bus,10.1,1,4
1,bus2,bus,12.1,2,5
2,bus3,bus,0.0,3,6


In [29]:
df_total = pd.concat([df_1, df_2], ignore_index = True)
df_total

,name,type,volts,current,x,y
0,load1,load,10.0,2.0,NaN,NaN
1,load2,load,12.0,1.8,NaN,NaN
2,bus1,bus,10.1,NaN,1.0,4.0
3,bus2,bus,12.1,NaN,2.0,5.0
4,bus3,bus,0.0,NaN,3.0,6.0


In [42]:
df_bus = df_total[df_total["type"] == "bus"].reset_index(drop = True)
df_bus["segment"] = df_bus.apply(lambda row: (row["x"], row["y"]) , axis = 1)
df_bus

,name,type,volts,current,x,y,segment
0,bus1,bus,10.1,NaN,1.0,4.0,"(1.0, 4.0)"
1,bus2,bus,12.1,NaN,2.0,5.0,"(2.0, 5.0)"
2,bus3,bus,0.0,NaN,3.0,6.0,"(3.0, 6.0)"


In [46]:
segment = df_bus["segment"].to_numpy()
segment

array([(1.0, 4.0), (2.0, 5.0), (3.0, 6.0)], dtype=object)

In [57]:
df_total.iloc[0].type

'load'